## Пример работы с данными реанализа ERA5 с помощью xarray

In [ ]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt

#### Загружаем датасет и добавляем в него скорость ветра

In [ ]:
ds = xr.load_dataset (r'../ERA5_demo_data/Moscow 2022.nc')


ds['vel10'] = np.sqrt (ds['u10']**2 + ds['v10']**2)
display(ds)


#### Выбираем данные за июнь и осредняем их по времени


In [ ]:
ds_cr = ds.where(ds['time'].dt.month == 6)
ds_mean = ds_cr.mean(dim = 'time')
ds_mean

#### Рисуем пространственное распределение осредненных значений

In [ ]:
(ds_mean['t2m']-273.15).plot()



#### Выбираем данные для ячейки, ближайшей к заданным координатам, и строим график

In [ ]:
ds4point = ds.sel (longitude = 37.5, latitude = 55.6, method = 'nearest')
display(ds4point)

ds4point['vel10'].plot()
ds4point['t2m'] = ds4point['t2m'] - 273.15

#### Конвертируем данные для точки в pandas DataFrame, чтобы можно было работать с ними так же, как с данными наблюдений ранее

In [ ]:
df4point = ds4point.to_dataframe()
display(df4point.head())

In [ ]:
#Загрузим данные наблюдений в центре Москвы
import weaclim_utils

df_urb = weaclim_utils.read_weaclim_dir ('weaclim_data/27605')


In [ ]:
rename = lambda df, word: df.rename(columns = {x:word + x for x in df.columns})

df = rename (df_urb, 'urb_').merge (rename (df4point, 'ERA5_'), left_index=True, right_index=True)
display(df.head())

df = df.sort_index()


In [ ]:
import plotly.express as px

fig = px.line(df, y=['urb_t2m', 'rur_t2m', 'ERA5_t2m'], title='Simple Time Series Plot') #, labels={'Value': 'Cumulative Value'})
fig.data[0].line.color = 'red'
fig.data[1].line.color = 'blue'
fig.data[2].line.color = 'black'
fig.show()